In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('data/train_resampled.csv')
test_df = pd.read_csv('data/test.csv').drop(columns='ID')

X_train = train_df.drop(columns='y')
y_train = train_df['y']

In [9]:
X_train.dtypes

x_0     float64
x_1     float64
x_2     float64
x_3     float64
x_4     float64
x_5      object
x_6     float64
x_7     float64
x_8     float64
x_9     float64
x_10    float64
dtype: object

In [2]:
def convert_to_float(x):
    try:
        return float(x)
    except ValueError:
        return np.nan
    
X_train_float = X_train.applymap(convert_to_float)

X_train_float_dropna = X_train_float.dropna()

/tmp/ipykernel_621841/221655602.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_train_float = X_train.applymap(convert_to_float)


In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_float_dropna)
X_train_scaled = pd.DataFrame(X_train_scaled)

In [4]:
from pycaret.regression import setup, compare_models, save_model, load_model, predict_model
from sklearn.metrics import mean_absolute_error


def run_automl(X, y):
    reg_setup = setup(data=pd.concat([X, y], axis=1), target='y', verbose=False, use_gpu=True, n_jobs=-1)
    top3_models = compare_models(sort='MSE', n_select=3)
    
    for idx, model in enumerate(top3_models):
        save_model(model, f'top{idx+1}_{model}')
    
    return top3_models

top3_models = run_automl(X_train_scaled, y_train)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more 

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,2.2426,8.6687,2.9387,0.5991,0.0354,0.0263,0.1890
rf,Random Forest Regressor,2.2656,8.9193,2.9813,0.5874,0.0359,0.0266,0.6010
lightgbm,Light Gradient Boosting Machine,2.5371,10.7784,3.2782,0.5010,0.0390,0.0296,0.2730
gbr,Gradient Boosting Regressor,2.6930,12.0508,3.4670,0.4419,0.0410,0.0314,2.2720
lr,Linear Regression,2.7847,12.6942,3.5584,0.4122,0.0419,0.0324,0.0120
br,Bayesian Ridge,2.7873,12.6958,3.5586,0.4121,0.0419,0.0325,0.0260
ridge,Ridge Regression,2.7960,12.7122,3.5609,0.4113,0.0420,0.0326,0.0120
huber,Huber Regressor,2.7295,12.9022,3.5877,0.4024,0.0423,0.0319,0.1680
knn,K Neighbors Regressor,2.7332,13.0691,3.6111,0.3944,0.0427,0.0319,0.0310
dt,Decision Tree Regressor,2.7846,15.2080,3.8882,0.2952,0.0483,0.0327,0.1530


Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved


In [5]:
for idx, model in enumerate(top3_models):
    print(model)
    
    test_scaled = scaler.transform(test_df)
    test_scaled = pd.DataFrame(test_scaled)
    preds = predict_model(model, data =test_scaled)
    
    submission =pd.read_csv('data/sample_submission.csv')
    submission['y']=preds['prediction_label']
    submission.to_csv(f'result_{model}.csv', index = False)

ExtraTreesRegressor(n_jobs=-1, random_state=4496)


RandomForestRegressor(n_jobs=-1, random_state=4496)


LGBMRegressor(device='gpu', n_jobs=-1, random_state=4496)
